In [160]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
plt.style.use('seaborn-whitegrid')

credit = pd.read_csv('https://raw.githubusercontent.com/jzuver/Final-211/main/BankChurners.csv')

def laplace_mech_vec(vec, sensitivity, epsilon):
    return [v + np.random.laplace(loc=0, scale=sensitivity / epsilon) for v in vec]


def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

custom_workload = [(len(credit[credit['Customer_Age'] > 10])), 
                     (len(credit[credit['Customer_Age'] > 45])),
                    (len(credit[(credit['Customer_Age'] > 40) & (credit['Gender'] == 'M')])),
                   (len(credit[(credit['Customer_Age'] > 40) & (credit['Gender'] == 'F')])),
                   (len(credit[(credit['Customer_Age'] < 40) & (credit['Gender'] == 'F')])),
                   (len(credit[(credit['Customer_Age'] == 40) & (credit['Education_Level'] == 'Graduate')])),
                   (len(credit[credit['Marital_Status'] == 'Single']) & len(credit[credit['Gender'] == 'M'])),
                   (len(credit[credit['Marital_Status'] == 'Single']) & len(credit[credit['Gender'] == 'F'])),
                   (len(credit[credit['Dependent_count'] == 4])),
                   (len(credit[credit['Card_Category'] ==  'Blue'])),
                   (len(credit[credit['Education_Level'] ==  'Uneducated'])),
                    (len(credit[credit['Credit_Limit'] >  10000])),
                   (len(credit[credit['Months_on_book'] >  36])),
                   (len(credit[(credit['Customer_Age'] < 30) & (credit['Education_Level'] == 'High School')])),
                         (len(credit[(credit['Customer_Age'] > 30) & (credit['Card_Category'] == 'Silver')]))
                  ]

In [161]:
credit.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.99991
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.99994
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.99998
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.99987
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.99998


In [162]:
real_answers = custom_workload

In [163]:
def workload_laplace_vec(workload, epsilon):
    # create parameters for range_query and list to store 
    noisy_vec_vals = laplace_mech_vec(workload, len(workload), epsilon)
    return noisy_vec_vals
print('First 4 answers:', workload_laplace_vec(custom_workload, 1.0)[:4])

First 4 answers: [10119.310227479365, 5506.853372091516, 3614.8526807568346, 4119.082729466954]


In [164]:
errors = [abs(r_a - l_a) for (r_a, l_a) in zip(real_answers, workload_laplace_vec(custom_workload, 1.0))]
print('Average absolute error:', np.mean(errors))

Average absolute error: 6.663162983759094


In [165]:
custom_workload

[10127,
 5466,
 3610,
 4120,
 1049,
 112,
 545,
 1126,
 1574,
 9436,
 1487,
 2754,
 3862,
 41,
 543]